# PointNet++: Deep Hierarchical Feature Learning on Point Sets in a Metric Space

## Paper Reviews

### Prior Reseach

**PointNet**은 Point Cloud data를 사용한 3D representation learner로서 pioneer적인 성공을 거두었음    
→ 그러나 irregular한 point set을 다뤄야 한다는 문제에 집중하여 몇가지 개선할 수 있는 room을 남겨뒀음    

+ Point Cloud sample들은 scan으로 얻어질 때 uniform하게 sample되지 않는 경우가 많은데 이에 대한 Architecture 차원의 고려가 없었음
+ 기존의 CNN에서 우수함을 보여줬던 hierarchical local feature learning을 위한 Architecture가 반영되지 않았음
+ 3D point라는 점에서 기본적으로 Euclidean space에 존재하므로 이를 이용하거나 또는 induced되는 metric을 이용할 수 있음

∴ PointNet Architecture를 baseline으로 point distribution에 robust하고 metric space를 활용하는 hierarchical point set feature learner인 **PointNet++** 를 제안

<p align="center">
  <img src="https://user-images.githubusercontent.com/86907286/193319945-262248a8-62db-4df6-b052-098625cefe32.png" alt="1" width="600px" />
</p>



### Set Abstraction

PointNet은 각 point들로 얻을 수 있는 global context를 위해 higher dimension feature로 mapping된 이 후 모든 point에 대해 **단 한번의 max pooling**이 이루어졌음  
이는 근본적으로 각 point의 feature representation이 **neighborhood의 정보를 반영하지 않고** summarize됨을 의미함  
→ hierarchical하게 local subset의 representation을 쌓아갈 수 있도록 **Set Abstraction**이라는 block을 통해 feature learning을 구현

Set Abstraction은 **Sampling Layer**, **Grouping Layer**, **PointNet Layer**로 구성됨  
이는 "local subset을 대표하는 sample point 선정 → local subset의 구축 → local subset 내부의 context learning"라는 과정으로 이해할 수 있음  

<p align="center">
  <img src="https://user-images.githubusercontent.com/86907286/193319950-493e27e0-7d7c-4f5a-962c-f68abfb91c85.png" alt="2" width="300px" />
</p>

#### Sampling Layer

Sampling Layer는 입력으로 주어지는 points $\{x_1, \cdots, x_n\}$에서 local subset의 centroid로 사용할 candidate를 찾는 과정임  
간단한 방법으로는 전체 $\{x_1, \cdots, x_n\}$에서 target centroid number만큼 uniform random sample하는 것임  
그러나 Point Cloud는 **항상 uniform하지 않기 때문**에 centroid들이 distinct context를 갖지 않고 하나의 semantic에 집중될 수 있음  

∴ 최대한 semantic이 잘 구분될 수 있는 point들을 sample 되도록 **Farthest Point Sampling**을 사용하여 최대한 sparse하게 subset을 추출  
= 이는 metric 측면에서 farthest로 sample되므로 non-euclidean metric을 고려해야 하는 경우에 대해서도 최적화될 수 있음 

<p align="center">
  <img src="https://user-images.githubusercontent.com/86907286/193319955-5e387725-b80b-4058-a44f-3d5a53faeed2.png" alt="3" width="400px" />
</p>

#### Grouping Layer

실제로 local representation을 얻을 subset에 대한 centroid가 sample되면 이와 함께 고려할 실질적인 subset의 element를 선정해야 함  
따라서 $N'$개의 centroid가 sample 되었다면 $N' \times K_{N'}$ 로 표현되는 subset들의 모음이 Grouping Layer의 output이 됨  

이때 일반적인 Clustering처럼 $K$를 fix하고 각 centroid가 갖는 point를 **K-Nearest Neighborhood**를 통해 선택할 수 있음  
그러나 Point Cloud는 non-uniform 할 수 있으므로 너무 sparse한 분포를 갖는 subset이 생성되어 local context가 파괴되는 형태가 발생할 수 있음  

∴ CNN의 kernel size처럼 fixed된 radius의 ball $ B_r(\hat{x}_i)$를 생각하고 내부에 있는 point를 하나의 subset으로 간주하는게 합리적임  
= 이 또한 metric space에서 induce되는 성질이므로 non-euclidean metric을 고려해야 하는 경우에 대해서 자유로운 구조  

#### PointNet

앞선 과정들은 모두 local set representation을 학습하기 위한 candidate 선정에 해당하므로 이에 대해서 summarize하는 과정이 반드시 필요  
따라서 선정된 subset point 내부에서의 context를 summarize하는 PointNet Architecture를 가져와 local representation을 학습   
→ 이때 centroid를 제외한 point는 centroid부터의 distance를 나타내도록 각 coordinate $j$에 대해서 $ x_i^{(j)} = x_i^{(j)} - \hat{x}^{(j)} $로 변환됨

### Robust Feature Learning under Non-Uniform Sampling Density

Architecture 차원에서 지속적으로 고려되듯이 Point Cloud는 non-uniform하게 분포하는 경우가 많아 이에 대해 robust해야 함  
이상적으로는 dense한 경우에서 fine-grained representation을 찾아낼 수 있도록 하는 것도 중요하다고는 할 수 있음  
그러나 해당 approach를 유지하기 위해서는 sampling 기반의 Point Cloud에서 low density 영역에 대해서 대처할 명확한 방법을 제시할 수 없음  

∴ 최대한 large scale representation부터 catch하여 sparse한 경우에서도 잘 수행할 수 있도록 구성하는게 합리적  
→ 이를 위해 **input 차원에서의 고려**와 **Architecture 차원에서의 고려** 두 방향으로 대처함 

input 차원으로는 사용되는 input point set의 일부를 drop하는 **random input dropout**을 적용함  
input으로 사용될 때마다 정해진 $p <= 1$를 따르는 $\text{Unif}(0, p)$에서 drop out ratio $\theta$를 얻어 해당 ratio 만큼 random drop을 수행  
→ 같은 3D object에 대해서 dense한 경우와 sparse한 경우 모두를 접할 수 있어 robust한 training을 유도할 수 있으며, empirical하게도 robust함이 관찰됨

<p align="center">
  <img src="https://user-images.githubusercontent.com/86907286/193319962-234445c4-93fd-4c5f-b1d5-37a3ac9631e0.png" alt="4" width="300px" />
</p>

Architecture 차원에서는 Set Abstraction을 확장한 **Density Adaptive PointNet Layer**를 적용할 수 있음  
vanilla인 경우의 Set Abstraction에서는 하나의 level은 원본 input point에서 해당 level이 담당하는 scale의 local ball 만을 참고하게 됨  
만약 input point가 sparse하다면 고정된 radius 범위를 참조하는 방법인 Set Abstraction은 local representation을 정상적으로 학습할 수 없음  
  
이는 hierarchical한 구조라도 sparse하면 이전 level에서 유의미한 representation을 catch하지 못했으므로 문제가 propagate 되는 형태일 뿐임  
→ 이를 위해 다양한 radius를 참조할 수 있도록 **Multi-scale Grouping(MSG)** 또는 **Multi-resolution Grouping(MRG)** 를 도입

<p align="center">
  <img src="https://user-images.githubusercontent.com/86907286/193319968-44f7768d-8e5a-481e-ae1d-bc0c27503f76.png" alt="5" width="300px" />
</p>


#### Multi-scale Grouping(MSG)

Multi-scale Grouping은 아주 단순하게 적용할 수 있는 방법의 adaptive grouping이라 할 수 있음  
Grouping Layer에서 사용하는 ball $ B_r(\hat{x}_i)$의 radius $r$을 여러 크기로 진행하여 이를 concatenate함  
이를 PointNet Layer로 넘겨주면 다양한 scale의 subset을 동시에 다룬 representation을 얻을 수 있음  

→ 단순하고 효과적이지만 모든 centroid에 이를 수행한다는 점에서 동시에 보는 scale이 늘어날 수록 연산량이 증가한다는 문제가 발생함

#### Multi-resolution Grouping(MRG)

Multi-scaled Grouping이 갖는 연산량의 증가를 막기 위한 대안으로서 좋은 방법  
이는 abstract하게 본다면 **이전 level에서의 grouping 정보를 raw하게 재활용하여** abstraction하는 것이라고 할 수 있음  
이전 level $L_{i-1}$에서 나온 feature point 만을 PointNet Layer로 summarize하는 것이 아니라 mapping 되기 전 raw input point 정보를 concatenate함 

→ 만약 dense하다면 이전 level에서 얻은 feature point, sparse하다면 raw point가 보완적으로 semantic을 보존해주므로 density에 robust 해질 수 있음

### Feature Propagation for Set Segmentation

PointNet++는 Set Abstraction을 위해서 centroid로 subset을 대표하는 과정을 거치므로 original point를 **subsampling**하게 됨  
그러나 segmentation을 수행하려면 **모든 input point마다 context를 얻어야 하므로** centroid를 중심으로 하는 context를 바로 이용할 수 없음   
이를 극복하기 위해 모든 input point를 centroid로 간주하고 수행한다면 연산량이 너무 커지므로 실현 불가능한 approach임  

∴ centroid에서 얻은 context를 **주변의 subset point로 propagate**하는 **Feature Propagation Level**을 추가로 도입



최종적으로 얻은 feature point로부터 $l$ level Set Abstraction의 raw input의 크기 $N_{l-1}$에 맞도록 **feature value를 interpolate**하여 subset point에 할당  
이렇게 얻은 feature vector들은 해당 level에 해당하는 $l-1$ level Set Abstraction로부터 skip-connection 되어 concatenate됨  
이후 CNN에서 1x1 Convolution처럼 coordinate가 아닌 각 feature dimension $ C $에 대해서만 수행되는 shared network인 **Unit PointNet**를 거침  
→ 이때 사용되는 interpolation 방법은 다음과 같은 **inverse distance weighted average based on K-nearest neighborhood** 임

$$ f^{j}(x) = \frac{\sum^k_{i=1} w_i(x)f_i^{(j)}}{\sum^k_{i=1} w_i(x)} \text{    where    } w_i(x)=\frac{1}{d(x, x_i)^p}, \,\, j = 1, \cdots, C $$


### Generalizability on Non-Euclidean Metric Space

Set Abstraction layer는 사용하고자 하는 metric space에서 induce되는 metric이 Euclidean metric이 아니어도 작동할 수 있음  
다음과 같이 특정한 **shape로 고정되지 않을 수 있는 non-rigid shape object**에 대해서는 Euclidean metric이 주는 context가 사라질 수 있음  
해당 경우는 직접 surface 상에서 생각하여 Non-Euclidean metric, 즉 곡면 상의 최단거리인 **geodesic distance**로 induce되는 metric의 사용이 적절함  

<p align="center">
  <img src="https://user-images.githubusercontent.com/86907286/193319978-51d6da94-2da5-44ae-8766-20d183634533.png" alt="6" width="300px" />
</p>

실제로 이러한 경우에서 Euclidean metric을 사용하는 것이 불리하다는 것을 empirical하게 확인할 수도 있었음  
하지만 PointNet++는 **Euclidean space에서가 아닌 일반적인 metric space에서의 distance로 정의될 수 있으므로** 해당 문제에 대해서 일반화가 가능함  
따라서 intrinsic structure를 반영할 수 있는 좋은 metric만 적절히 주어진다면 이 위에서 좋은 representation을 학습할 수 있음

<p align="center">
  <img src="https://user-images.githubusercontent.com/86907286/193319986-09ca50ac-a20e-4b93-a1bd-8d1b511b683c.png" alt="7" width="500px" />
</p>



## Implementation Reviews

Set Abstraction base

In [ ]:
class _PointnetSAModuleBase(nn.Module):

    def __init__(self):
        super().__init__()
        self.npoint = None
        self.groupers = None
        self.mlps = None
        self.pool_method = 'max_pool'

    def forward(self, xyz: torch.Tensor, features: torch.Tensor = None, new_xyz=None) -> (torch.Tensor, torch.Tensor):
        """
        :param xyz: (B, N, 3) tensor of the xyz coordinates of the features
        :param features: (B, N, C) tensor of the descriptors of the the features
        :param new_xyz:
        :return:
            new_xyz: (B, npoint, 3) tensor of the new features' xyz
            new_features: (B, npoint, \sum_k(mlps[k][-1])) tensor of the new_features descriptors
        """
        new_features_list = []

        xyz_flipped = xyz.transpose(1, 2).contiguous()
        if new_xyz is None:
            new_xyz = pointnet2_utils.gather_operation(
                xyz_flipped,
                pointnet2_utils.furthest_point_sample(xyz, self.npoint)
            ).transpose(1, 2).contiguous() if self.npoint is not None else None

        for i in range(len(self.groupers)):
            new_features = self.groupers[i](xyz, new_xyz, features)  # (B, C, npoint, nsample)

            new_features = self.mlps[i](new_features)  # (B, mlp[-1], npoint, nsample)
            if self.pool_method == 'max_pool':
                new_features = F.max_pool2d(
                    new_features, kernel_size=[1, new_features.size(3)]
                )  # (B, mlp[-1], npoint, 1)
            elif self.pool_method == 'avg_pool':
                new_features = F.avg_pool2d(
                    new_features, kernel_size=[1, new_features.size(3)]
                )  # (B, mlp[-1], npoint, 1)
            else:
                raise NotImplementedError

            new_features = new_features.squeeze(-1)  # (B, mlp[-1], npoint)
            new_features_list.append(new_features)

        return new_xyz, torch.cat(new_features_list, dim=1)

Multi-scale Grouping

In [ ]:
class PointnetSAModuleMSG(_PointnetSAModuleBase):
    """Pointnet set abstraction layer with multiscale grouping"""

    def __init__(self, *, npoint: int, radii: List[float], nsamples: List[int], mlps: List[List[int]], bn: bool = True,
                 use_xyz: bool = True, pool_method='max_pool', instance_norm=False):
        """
        :param npoint: int
        :param radii: list of float, list of radii to group with
        :param nsamples: list of int, number of samples in each ball query
        :param mlps: list of list of int, spec of the pointnet before the global pooling for each scale
        :param bn: whether to use batchnorm
        :param use_xyz:
        :param pool_method: max_pool / avg_pool
        :param instance_norm: whether to use instance_norm
        """
        super().__init__()

        assert len(radii) == len(nsamples) == len(mlps)

        self.npoint = npoint
        self.groupers = nn.ModuleList()
        self.mlps = nn.ModuleList()
        for i in range(len(radii)):
            radius = radii[i]
            nsample = nsamples[i]
            self.groupers.append(
                pointnet2_utils.QueryAndGroup(radius, nsample, use_xyz=use_xyz)
                if npoint is not None else pointnet2_utils.GroupAll(use_xyz)
            )
            mlp_spec = mlps[i]
            if use_xyz:
                mlp_spec[0] += 3

            self.mlps.append(pt_utils.SharedMLP(mlp_spec, bn=bn, instance_norm=instance_norm))
        self.pool_method = pool_method

Feature Propagating

In [ ]:
class PointnetFPModule(nn.Module):
    r"""Propigates the features of one set to another"""

    def __init__(self, *, mlp: List[int], bn: bool = True):
        """
        :param mlp: list of int
        :param bn: whether to use batchnorm
        """
        super().__init__()
        self.mlp = pt_utils.SharedMLP(mlp, bn=bn)

    def forward(
            self, unknown: torch.Tensor, known: torch.Tensor, unknow_feats: torch.Tensor, known_feats: torch.Tensor
    ) -> torch.Tensor:
        """
        :param unknown: (B, n, 3) tensor of the xyz positions of the unknown features
        :param known: (B, m, 3) tensor of the xyz positions of the known features
        :param unknow_feats: (B, C1, n) tensor of the features to be propigated to
        :param known_feats: (B, C2, m) tensor of features to be propigated
        :return:
            new_features: (B, mlp[-1], n) tensor of the features of the unknown features
        """
        if known is not None:
            dist, idx = pointnet2_utils.three_nn(unknown, known)
            dist_recip = 1.0 / (dist + 1e-8)
            norm = torch.sum(dist_recip, dim=2, keepdim=True)
            weight = dist_recip / norm

            interpolated_feats = pointnet2_utils.three_interpolate(known_feats, idx, weight)
        else:
            interpolated_feats = known_feats.expand(*known_feats.size()[0:2], unknown.size(1))

        if unknow_feats is not None:
            new_features = torch.cat([interpolated_feats, unknow_feats], dim=1)  # (B, C2 + C1, n)
        else:
            new_features = interpolated_feats

        new_features = new_features.unsqueeze(-1)
        new_features = self.mlp(new_features)

        return new_features.squeeze(-1)

## Reference

https://arxiv.org/abs/1706.02413  
https://github.com/sshaoshuai/Pointnet2.PyTorch/blob/master/pointnet2/pointnet2_modules.py